In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from joblib import load
from sklearn.preprocessing import StandardScaler


def extract_team_nickname(full_name):
    return full_name.split()[-1]

def format_features_for_prediction(team_name, team_stats, is_home_game):
    """
    Format the latest stats into the feature array expected by the model.

    :param team_name: Name of the team.
    :param team_stats: DataFrame with the latest stats for the team.
    :param is_home_game: Boolean indicating if the team is playing at home in the upcoming game.
    :return: List of features in the correct order for model prediction.
    """
    if team_stats.empty:
        print(f"Stats DataFrame is empty for team: {team_name}")
        return None

    try:
        # Determine the prefix based on whether the team is playing at home or away
        prefix = 'home' if is_home_game else 'visitor'

        # Extract relevant stats using the appropriate prefix
        features = [
            team_stats[f'ast_{prefix}_10game_avg'],
            team_stats[f'blk_{prefix}_10game_avg'],
            team_stats[f'dreb_{prefix}_10game_avg'],
            team_stats[f'fg3_pct_{prefix}_10game_avg'],
            team_stats[f'fg_pct_{prefix}_10game_avg'],
            team_stats[f'ft_pct_{prefix}_10game_avg'],
            team_stats[f'oreb_{prefix}_10game_avg'],
            team_stats[f'pf_{prefix}_10game_avg'],
            team_stats[f'pts_{prefix}_10game_avg'],
            team_stats[f'reb_{prefix}_10game_avg'],
            team_stats[f'stl_{prefix}_10game_avg'],
            team_stats[f'turnover_{prefix}_10game_avg'],        
            team_stats[f'ast_{prefix}_season_avg'],
            team_stats[f'blk_{prefix}_season_avg'],
            team_stats[f'dreb_{prefix}_season_avg'],
            team_stats[f'fg3_pct_{prefix}_season_avg'],
            team_stats[f'fg_pct_{prefix}_season_avg'],
            team_stats[f'ft_pct_{prefix}_season_avg'],
            team_stats[f'oreb_{prefix}_season_avg'],
            team_stats[f'pf_{prefix}_season_avg'],
            team_stats[f'pts_{prefix}_season_avg'],
            team_stats[f'reb_{prefix}_season_avg'],
            team_stats[f'stl_{prefix}_season_avg'],
            team_stats[f'turnover_{prefix}_season_avg'],        
        ]

        return features
    except Exception as e:
        print(f"Error in formatting features for {team_name}: {e}")
        return None




def find_latest_team_data_by_name(team_name, game_stats_df, is_home_team_upcoming):
    team_name = team_name.lower()

    team_games = game_stats_df[
        (game_stats_df['team_name_home'].str.lower().str.contains(team_name)) |
        (game_stats_df['team_name_visitor'].str.lower().str.contains(team_name))
    ]

    if not team_games.empty:
        latest_game_date = team_games['game_date'].max()
        latest_game_data = team_games[team_games['game_date'] == latest_game_date]

        if not latest_game_data.empty:
            home_team_name = latest_game_data.iloc[0]['team_name_home'].lower()
            is_home_team = team_name in home_team_name
            prefix = 'home' if is_home_team else 'visitor'
            rename_prefix = 'home' if is_home_team_upcoming else 'visitor'

            # Fetching the columns and renaming based on whether the team is home or visitor
            stats_columns = [col for col in game_stats_df.columns if f'{prefix}_' in col and ('10game_avg' in col or 'season_avg' in col)]
            if stats_columns:
                latest_team_stats = latest_game_data[stats_columns].iloc[0]

                # Rename columns by adding the correct prefix
                renamed_stats = latest_team_stats.rename(lambda x: x.replace(f'{prefix}_', f'{rename_prefix}_'))
                
                return pd.DataFrame([renamed_stats])
            else:
                print(f"No stats columns found for prefix: {prefix}")  # Debug print

    print(f"No data found for team: {team_name}")  # Debug print
    return pd.DataFrame()


# Load the NBA schedule, game statistics, and the trained model
schedule_df = pd.read_csv('./nba_schedule.csv')
game_stats_df = pd.read_csv('./game_stats.csv')  # Replace with the correct path
model_pipeline = load('nba_game_predictor_model.joblib')

schedule_df = pd.read_csv('./nba_schedule.csv')

# Convert date columns to datetime and adjust team names
schedule_df['date'] = pd.to_datetime(schedule_df['date'])
schedule_df['Home Team'] = schedule_df['Home Team'].apply(extract_team_nickname)
schedule_df['Visitor Team'] = schedule_df['Visitor Team'].apply(extract_team_nickname)

# Convert date columns to datetime
schedule_df['date'] = pd.to_datetime(schedule_df['date'])
game_stats_df['game_date'] = pd.to_datetime(game_stats_df['game_date'])

# Filter games for today
today = datetime.now().strftime('%Y-%m-%d')
todays_games = schedule_df[schedule_df['date'] == today]

print(f"Found {len(todays_games)} games today.")
print(todays_games[['date', 'Home Team', 'Visitor Team']])

feature_columns = [
    'ast_home_10game_avg', 'blk_home_10game_avg', 'dreb_home_10game_avg', 
    'fg3_pct_home_10game_avg', 'fg_pct_home_10game_avg', 'ft_pct_home_10game_avg', 
    'oreb_home_10game_avg', 'pf_home_10game_avg', 'pts_home_10game_avg', 
    'reb_home_10game_avg', 'stl_home_10game_avg', 'turnover_home_10game_avg',
    'ast_visitor_10game_avg', 'blk_visitor_10game_avg', 'dreb_visitor_10game_avg', 
    'fg3_pct_visitor_10game_avg', 'fg_pct_visitor_10game_avg', 'ft_pct_visitor_10game_avg', 
    'oreb_visitor_10game_avg', 'pf_visitor_10game_avg', 'pts_visitor_10game_avg', 
    'reb_visitor_10game_avg', 'stl_visitor_10game_avg', 'turnover_visitor_10game_avg',
    'ast_visitor_season_avg', 'blk_visitor_season_avg', 'dreb_visitor_season_avg', 
    'fg3_pct_visitor_season_avg', 'fg_pct_visitor_season_avg', 'ft_pct_visitor_season_avg', 
    'oreb_visitor_season_avg', 'pf_visitor_season_avg', 'pts_visitor_season_avg', 
    'reb_visitor_season_avg', 'stl_visitor_season_avg', 'turnover_visitor_season_avg'
    # ... make sure this order is exactly the same as in the training dataset
]

# Prepare features for each game and make predictions
predictions = []
for index, upcoming_game in todays_games.iterrows():
    home_team_name = upcoming_game['Home Team']
    visitor_team_name = upcoming_game['Visitor Team']

    home_team_stats = find_latest_team_data_by_name(home_team_name, game_stats_df, is_home_team_upcoming=True)
    visitor_team_stats = find_latest_team_data_by_name(visitor_team_name, game_stats_df, is_home_team_upcoming=False)

    if home_team_stats is not None and visitor_team_stats is not None:
        # Combine home and visitor features
        game_features = []
        for feature in feature_columns:
            if feature in home_team_stats.columns:
                game_features.append(home_team_stats[feature].iloc[0])
            elif feature in visitor_team_stats.columns:
                game_features.append(visitor_team_stats[feature].iloc[0])
            else:
                game_features.append(np.nan)  # Or a suitable default value

        # Convert to DataFrame with correct column names
        game_features_df = pd.DataFrame([game_features], columns=feature_columns)

        # Make prediction
        prediction = model_pipeline.predict(game_features_df)
        predicted_winner = home_team_name if prediction[0] == 1 else visitor_team_name
        predictions.append(predicted_winner)
    else:
        predictions.append("Unknown")


# Use .loc to avoid SettingWithCopyWarning
todays_games.loc[:, 'predicted_winner'] = predictions

# Output the predictions
print(todays_games[['date', 'Home Team', 'Visitor Team', 'predicted_winner']])

Found 0 games today.
Empty DataFrame
Columns: [date, Home Team, Visitor Team]
Index: []
Empty DataFrame
Columns: [date, Home Team, Visitor Team, predicted_winner]
Index: []
